<a href="https://colab.research.google.com/github/Vinhnv0901/student_project/blob/master/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

#load data

In [ ]:
!git clone "https://github.com/Vinhnv0901/student_project.git"

In [ ]:
data_grade = pd.read_excel("/content/student_project/Education_dataset_V2/02.diem.xlsx")
data_inf = pd.read_excel("/content/student_project/Education_dataset_V2/01.sinhvien.xlsx")

In [ ]:
data_inf = data_inf[['mssv', ' namsinh', ' gioitinh', ' noisinh', ' lopsh', ' khoa', ' hedt', ' khoahoc', ' chuyennganh2',' tinhtrang']]
data_inf.columns = ['mssv', 'namsinh', 'gioitinh', 'noisinh', 'lopsh', 'khoa', 'hedt', 'khoahoc', 'chuyennganh2','tinhtrang']
data_grade = data_grade[['mssv', ' mamh', ' malop', ' sotc', ' namhoc', ' hocky', ' diem', ' trangthai', ' mamh_tt']]
data_grade.columns = ['mssv', 'mamh', 'malop', 'sotc', 'namhoc', 'hocky', 'diem', 'trangthai', 'mamh_tt']

#merge student table and grade table

In [ ]:
data_merge = pd.merge(data_grade, data_inf, on = 'mssv', how = 'inner')

In [ ]:
print('the number of nan value each columns:',{col: list(pd.isna(data_merge[col])).count(True) for col in data_merge.columns})

#clean data

In [ ]:
data_merge = data_merge.dropna(subset = ['diem'])#xóa các dòng có giá trị nan trong cột điểm 
data_merge['diem'] = data_merge['diem'].replace(' NULL', np.nan)#chuyển đổi NUll về nan để thêm giá trị
data_merge['diem'] = data_merge['diem'].fillna(method = 'ffill')# thêm giá trị điểm có các dùng nan
data_merge = data_merge.astype(dtype = {'namhoc': np.int32, 'diem': np.float32})
data_merge = data_merge.drop(data_merge[data_merge['namhoc'] == 2012].index)#xóa các dòng năm 2012 vì dữ liệu quá ít
#khóa khoảng trắng các cột mamh, khoa, khoaMH
data_merge['mamh'] = data_merge['mamh'].str.strip()
data_merge['khoa'] = data_merge['khoa'].str.strip()
data_merge = data_merge.reset_index()

#add column cate of course

In [ ]:
#Xác định môn học thuộc khoa nào
mamh = data_merge['mamh'].copy()
for i in range(data_merge.shape[0]):
  mamh[i] = mamh[i].translate(str.maketrans("","", '0123456789'))
a = mamh.unique() #các mã môn học đại diện cho khoa quản lý
b = ['MHKT', 'KTMT', 'HTTT', 'HTTT', 'PĐTĐH', 'BMTL', 'KHMT', 'PĐTĐH', 'HTTT', 'HTTT', 'BMAV', 'MMT&TT', 'CNPM', 'BMAV', 'HTTT', 'KTTT', 'HTTT', 'KTMT', 'CNPM', 'HTTT', 'HTTT', 'HTTT','HTTT', 'PĐTĐH', 'PĐTĐH'] # các khoa
mamh.replace(to_replace = a, value = b, inplace = True)
for i in range(data_merge.shape[0]):#các nhóm môn học có của cái đầu là 'IT' cần được điều chỉnh lại
  ma = data_merge.loc[i,'mamh']
  if ma == 'IT001' or ma == 'IT003' or ma == 'IT011' or ma == 'IT013':
    mamh[i] = 'KHMT'
  elif ma == 'IT002' or ma == 'IT008':
    mamh[i] = 'CNPM'
  elif ma == 'IT004':
    mamh[i] = 'HTTT'
  elif ma == 'IT005':
    mamh[i] = 'MMT&TT'
  elif ma == 'IT006' or ma == 'IT007' or ma =='IT010' or ma == 'IT012':
    mamh[i] = 'KTMT'
  elif ma == 'IT009':
    mamh[i] = 'PĐTĐH'
data_merge['KhoaMH'] = np.array(mamh)

#remove third semester of the year

In [ ]:

data_merge = data_merge.drop(data_merge[data_merge['hocky'] == 3].index)#xóa các dòng chứa học kỳ hè vì nó chỉ chiếm 2% trong dataset
data_merge = data_merge.reset_index()

#add column semester of each enroll

In [ ]:
#Đầu tiên xác định xem sinh viên nhập học năm mấy
khoaHoc = data_merge['khoahoc'].copy()
lis = np.unique(khoaHoc)
lis_replace = np.array([2013, 2014, 2015, 2016])
khoaHoc.replace(to_replace = lis, value = lis_replace, inplace = True)
data_merge['namNhapHoc'] = khoaHoc
#Tính học kỳ của của từng đăng ký
data_merge['soHocKy'] = (data_merge['namhoc'] - data_merge['namNhapHoc'])*2 + data_merge['hocky']

#Divide train and test:
##80% students train and 20% students test
##recommend courses of two terms in 2016

In [ ]:
listStudent = np.unique(data_merge['mssv'])
student_train, student_test = train_test_split(listStudent, test_size=0.2, random_state=42)

data_train = data_merge[data_merge['mssv'].isin(student_train)]
data_test = data_merge[data_merge['mssv'].isin(student_test) & data_merge['namhoc'].isin([2013, 2014, 2015])]